Combining all three

In [4]:
import numpy as np
from qiskit import *
from qiskit_ibm_runtime import QiskitRuntimeService
from ExperimentClass import Experiment
import json
import sys
import os

In [ ]:
service = QiskitRuntimeService(channel="ibm_quantum", 
                                  token="<Your-Token-goes-here>")
backend = service.backend("ibm_brisbane")

In [5]:
# Configuration file name
CONFIG_FILE = "config.json"

# Default configuration template
DEFAULT_CONFIG = {
    "numOfAttacks": None,
    "initialLayout": None,
    "initialLayoutWithBuffer": None,
    "initialState": None,
    "ddSequenceType": None
}

def create_config(file_path):
    """Create a config file with expected keys set to None if it doesn't exist."""
    with open(file_path, "w") as file:
        json.dump(DEFAULT_CONFIG, file, indent=4)
    print(f"Configuration file '{file_path}' was missing and has been created.")
    print("Please update it with appropriate values before running the script again.")
    sys.exit(1)

def load_config(file_path):
    """Load configuration from a JSON file, or create it if missing."""
    if not os.path.exists(file_path):
        create_config(file_path)

    try:
        with open(file_path, "r") as file:
            config = json.load(file)
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in '{file_path}'. Please check your file.")
        sys.exit(1)
    
    return config

def validate_config(config):
    """Validate the configuration parameters."""
    for key in DEFAULT_CONFIG.keys():
        if key not in config or config[key] is None:
            print(f"Error: Missing or null value for '{key}' in '{CONFIG_FILE}'.")
            print("Please update the config file and re-run the script.")
            sys.exit(1)
    
    numOfAttacks = config["numOfAttacks"]
    if not isinstance(numOfAttacks, int) or numOfAttacks < 1:
        print("Error: 'numOfAttacks' must be a positive integer (at least 1).")
        sys.exit(1)

    numQubits = 3 + numOfAttacks * 2
    expectedBufferQubits = 3 + numOfAttacks * 3

    if not isinstance(config['initialLayout'],list):
        print(f"Error: 'initialLayout' must be an array.")
        sys.exit(1)
    elif len(config["initialLayout"]) != numQubits:
        print(f"Error: 'initialLayout' must contain {numQubits} qubits but found {len(config['initialLayout'])}.")
        sys.exit(1)
    
    if not isinstance(config['initialLayoutWithBuffer'],list):
        print(f"Error: 'initialLayoutWithBuffer' must be an array.")
        sys.exit(1)
    elif len(config["initialLayoutWithBuffer"]) != expectedBufferQubits:
        print(f"Error: 'initialLayoutWithBuffer' must contain {expectedBufferQubits} qubits but found {len(config['initialLayoutWithBuffer'])}.")
        sys.exit(1)

    if config["initialState"] not in [0, 1, 2]:
        print("Error: 'initialState' must be 0 (|0>), 1 (|1>), or 2 (|+>).")
        sys.exit(1)

    if config["ddSequenceType"] not in [0, 1]:
        print("Error: 'ddSequenceType' must be 0 (XYXY) or 1 (XX).")
        sys.exit(1)

In [ ]:
# Load and validate configuration file
print("Loading experiment configuration...")
config = load_config(CONFIG_FILE)
validate_config(config)

# Display loaded configuration parameters
print("\nConfiguration Loaded Successfully!")
print(f"Total Number of Attacks: {config['numOfAttacks']}")
print(f"Initial Layout (without buffer): {config['initialLayout']}")
print(f"Initial Layout (with buffer): {config['initialLayoutWithBuffer']}")
print(f"Initial State of Control Qubit: {config['initialState']}")
print(f"DD Sequence Type: {config['ddSequenceType']}")

Loading experiment configuration...

Configuration Loaded Successfully!
Total Number of Attacks: 3
Initial Layout (without buffer): [4, 3, 5, 2, 1, 6, 7, 15, 22]
Initial Layout (with buffer): [4, 3, 5, 2, 1, 0, 6, 7, 8, 15, 22, 23]
Initial State of Control Qubit: 0
DD Sequence Type: 0


In [ ]:
# Initialize Experiment with required parameters
numQubits = config['numOfAttacks'] * 2 + 3
experiment = Experiment(numQubits,
                        config['initialLayout'],
                        config['initialLayoutWithBuffer'],
                        backend,
                        config['initialState'],
                        config['ddSequenceType'])

# Define scenarios to be tested
scenarios = [
        ("No Attack scenario", experiment.addNoAttackCircuit),
        ("No Attack with DD Scenario", experiment.addNoAttackPlusDDCircuit),
        ("Attack without Mitigation Scenario", experiment.addAttackWithoutMitigationCircuits),
        ("Attack with DD Scenario", experiment.addAttackWithDDCircuits),
        ("Attack with Spacing Scenario", experiment.addAttackWithSpacingCircuits),
        ("Attack with DD and Spacing Scenario", experiment.addAttackWithDDAndSpacingCircuits)
    ]

# Loop through scenarios based on user input
for scenario_name, scenario_func in scenarios:
    choice = input(f"Do you want to test code for {scenario_name}? Yes or No: ").strip().lower()
    if choice == "yes":
        print(f"Add circuits for {scenario_name}")
        scenario_func()
        print(f"Circuits for {scenario_name} added.")
    elif choice == "no":
        print(f"Skipping {scenario_name}...")
    else:
        print("Invalid input! Skipping...")

Add circuits for No Attack scenario
Circuits for No Attack scenario added.
Add circuits for No Attack with DD Scenario
Circuits for No Attack with DD Scenario added.
Add circuits for Attack without Mitigation Scenario
Circuits for Attack without Mitigation Scenario added.
Add circuits for Attack with DD Scenario
Circuits for Attack with DD Scenario added.
Add circuits for Attack with Spacing Scenario
Circuits for Attack with Spacing Scenario added.
Add circuits for Attack with DD and Spacing Scenario
Circuits for Attack with DD and Spacing Scenario added.


In [ ]:
# Run all experiments and collect results
experiment.runAllCircuits()

In [ ]:
# Calculate and retrieve values of fidelities
experiment.calculateFidelityOfDataQubits()
fidelities = experiment.getFidelities()

In [ ]:
# Plot the results
# Modify the plots according to the experiment scenarios selected.
import matplotlib.pyplot as plt

x = range(45)

plt.figure(figsize=(8, 6))

plt.scatter(range(1), fidelities[0], linestyle='-', color='teal', label="Free Evolution")
plt.scatter(range(1), fidelities[1], linestyle='-', color='orange', label="No Attack + DD")
plt.plot(x, fidelities[2:47], linestyle='-', color='green', label="Attack without Mitigation")
plt.plot(x, fidelities[47:92], linestyle='-', color='blue', label="Attack + DD")
plt.plot(x, fidelities[92:137], linestyle='-', color='magenta', label="Attack + Spacing")
plt.plot(x, fidelities[137:182], linestyle='-', color='brown', label="Attack + Spacing & DD")

plt.title('Line Chart of Given Data')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Write experiment raw result (counts) to an external file for later reference
def append_to_json_file(file_path, new_data):
    """ Open the file in append mode """
    with open(file_path, 'a') as file:
        """ Serialize the new data to a JSON formatted string """
        json_data = json.dumps(new_data)
        """ Write the JSON data to the file, followed by a newline """
        file.write(json_data + '\n')

if len(experiment.getResult()):
    """ Append the new data to the file """
    append_to_json_file('ExperimentResult_Count.json', experiment.getResult())

if len(fidelities):
    """ Append the new data to the file """
    append_to_json_file('ExperimentResult_Fidelities.json', fidelities)